# STEEL SLITTING OPTIMA

### I Problem Statement

The Cutting Stock Problem deals with the problem of cutting stock material with the same, fixed width — such as paper rolls — into smaller pieces, according to a set of orders specifying both the widths and the demand requirements, so as to minimize the amount of wasted material.

![image](cutting_stock.png)

-----

To choose the *mother coil* from the stock to cut into order in PO received from clients. To minimize the trim loss from slitting the coil

**Key terms**
Need_cut = weight \\
Remain_Weight 
Trim_Loss_Weight 
Trim_Loss 
Num_Split


```python
if Thickness <= 3 and Thickness > 1:
        min_loss = 8
    elif Thickness == 1:
        min_loss = 5.5
    else:
        min_loss = 3*Thickness      
    max_loss= int(0.04*Width)

# Hyperpara
stock_name = 'HSC'	
Thickness = 2.6
Spec =  'JSH270C-PO' 

# Optimiza Parameter'
optima_ratio_fc1 = 50%
# optima_ratio_fc1 = 200%

In [1]:
import pandas as pd
import numpy as np

# IMPORT SAMPLE DATA
sample_po = pd.read_pickle("data/sample_po.pkl")
sample_coil = pd.read_pickle("data/sample_coils_instock.pkl")

sample_po['Width'] = sample_po['Width'].astype('int')
df = sample_po.groupby('Width').size().reset_index(name='Count')
df

,Width,Count
0,52,2
1,70,1
2,72,1
3,106,1
4,120,1
5,127,1
6,145,1
7,150,1
8,220,1


In [ ]:
sample_coil

In [ ]:

# Convert DataFrame to dictionary
orders = df.to_dict(orient='index')

# Remove column names from values
for key, value in orders.items():
    orders[key] = list(value.values())
orders

## Problem Formulation

Consider a set ${S}$ of available stock materials that can be cut to size to produce a set of finished parts. Each stock $s\in S$ is characterized by a width $wd^S_s$, a weight $wt_s$ per piece, and is available in *limited* quantity. A customer order is received to product a set of finished products $F$. Each finished product $f\in F$ is specified by a required weight $wt_f$ and width $wd^F_f$.

The **cutting stock problem** is to find a minimum waste to fulfill the customer order from the stock materials.

**Stocks**

| stocks <br> $s$ | width <br> $wd^S_s$ | weight <br> $wt_s$ |
| :--: | :--: | :--: |
| A | 5 | 6 |
| B | 6 | 7 |
| C | 9 |10 |

**Finished Parts**

| finished parts <br> $f$ | width <br> $wd^F_f$ | weight <br> $wt_f$ |
| :--: | :--: | :--: |
| S | 2 | 20 |
| M | 3 | 10 |
| L | 4 | 20 |


In [2]:
stocks = {
    "A": {"Width": 1219, "weight": 4000},
    "B": {"Width": 1219, "weight": 400},
    "C": {"Width": 1108, "weight": 4266},
    "D": {"Width": 1219, "weight": 5350},
}

finish = {
    "XXL": {"Width": 220, "need_cut": -4247 , "fc1":22574},
    "L": {"Width": 145, "need_cut": -1895, "fc1": 4559},
    "XL": {"Width": 150, "need_cut": -788, "fc1": 4618},
    "M": {"Width": 70, "need_cut": -515, "fc1": 669},
    "S": {"Width": 52, "need_cut": -400, "fc1": 0},
}

## Patterns - Problem Space

One approach to solving this problem is to create a list of all finished parts, a list of stocks for each length, and then use a set of binary decision variables to assign each finished product to a particular piece of stock. This approach will work well for a small problems, but the computational complexity scales much too rapidly with the size of the problem to be practical for business applications.

A pattern is a list of finished parts that can be cut from a particular stock item.

A pattern $p$ is specified by the stock $s_p$ assigned to the pattern and integers $cut_{pf}$ that specify how many finished parts of type $f$ are cut from stock $s_p$. A pattern $p\in P$ is feasible if

$$
\begin{align}
\sum_{f\in F}cut_{pf}wd^F_f  & \leq   wd^S_{s_p}
\end{align}
$$

The function `make_patterns` defined below produces a partial list of feasible patterns for given sets of stocks and finished parts. Each pattern is represented as dictionary that specifies an associated stock item, and a dictionary of cuts that specify the finished parts cut from the stock. The algorithm is simple, it just considers every finished parts and stock items, then reports the number of parts $f$ that can be cut from stock item $s$.

A mathematical optimization model has five components, namely:

- Sets and indices.
- Parameters.
- Decision variables.
- Objective function(s).
- Constraints.

In [ ]:
class MasterProblem:
    def __init__(self):
        self.model = gp.Model("master")
        self.vars = None
        self.constrs = None
        
    def setup(self, patterns, demand):
        num_patterns = len(patterns)
        self.vars = self.model.addVars(num_patterns, obj=1, name="Pattern")
        self.constrs = self.model.addConstrs((gp.quicksum(patterns[pattern][piece]*self.vars[pattern]
                                                          for pattern in range(num_patterns))
                                              >= demand[piece] for piece in demand.keys()),
                                             name="Demand")
        self.model.modelSense = GRB.MINIMIZE
        # Turning off output because of the iterative procedure
        self.model.params.outputFlag = 0
        self.model.update()
        
    def update(self, pattern, index):
        new_col = gp.Column(coeffs=pattern, constrs=self.constrs.values())
        self.vars[index] = self.model.addVar(obj=1, column=new_col,
                                             name=f"Pattern[{index}]")
        self.model.update()


class SubProblem:
    def __init__(self):
        self.model = gp.Model("subproblem")
        self.vars = {}
        self.constr = None
        
    def setup(self, stock_length, lengths, duals):
        self.vars = self.model.addVars(len(lengths), obj=duals, vtype=GRB.INTEGER,
                                       name="Frequency")
        self.constr = self.model.addConstr(self.vars.prod(lengths) <= stock_length,
                                           name="Knapsack")
        self.model.modelSense = GRB.MAXIMIZE
        # Turning off output because of the iterative procedure
        self.model.params.outputFlag = 0
        # Stop the subproblem routine as soon as the objective's best bound becomes
        #less than or equal to one, as this implies a non-negative reduced cost for
        #the entering column.
        self.model.params.bestBdStop = 1
        self.model.update()
        
    def update(self, duals):
        self.model.setAttr("obj", self.vars, duals)
        self.model.update()


class CuttingStock:
    def __init__(self, stock_length, pieces):
        self.stock_length = stock_length
        self.pieces, self.lengths, self.demand = gp.multidict(pieces)
        self.patterns = None
        self.duals = [0]*len(self.pieces)
        piece_reqs = [length*req for length, req in pieces.values()]
        self.min_rolls = np.ceil(np.sum(piece_reqs)/stock_length)
        self.solution = {}
        # self.master = MasterProblem()
        # self.subproblem = SubProblem()
        
    def _initialize_patterns(self):
        # Find trivial patterns that consider one final piece at a time,
        #fitting as many pieces as possible into the stock material unit
        patterns = []
        for idx, length in self.lengths.items():
            pattern = [0]*len(self.pieces)
            pattern[idx] = self.stock_length // length
            patterns.append(pattern)
        self.patterns = patterns
        
    def _generate_patterns(self):
        self._initialize_patterns()
        self.master.setup(self.patterns, self.demand)
        self.subproblem.setup(self.stock_length, self.lengths, self.duals)
        while True:
            self.master.model.optimize()
            self.duals = self.master.model.getAttr("pi", self.master.constrs)
            self.subproblem.update(self.duals)
            self.subproblem.model.optimize()
            reduced_cost = 1 - self.subproblem.model.objVal
            if reduced_cost >= 0:
                break
            
            pattern = [0]*len(self.pieces)
            for piece, var in self.subproblem.vars.items():
                if var.x > 0.5:
                    pattern[piece] = round(var.x)
            self.master.update(pattern, len(self.patterns))
            self.patterns.append(pattern)

In [3]:
def make_patterns(stocks, finish):
    """
    Generates patterns of feasible cuts from stock lengths to meet specified finish lengths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts = int(stocks[s]["Width"] / finish[f]["Width"])

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for {f}")
            return []

    return patterns


patterns = make_patterns(stocks, finish)
display(patterns)

[{'stock': 'A', 'cuts': {'XXL': 5, 'L': 0, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'B', 'cuts': {'XXL': 5, 'L': 0, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'C', 'cuts': {'XXL': 5, 'L': 0, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'D', 'cuts': {'XXL': 5, 'L': 0, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'A', 'cuts': {'XXL': 0, 'L': 8, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'B', 'cuts': {'XXL': 0, 'L': 8, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'C', 'cuts': {'XXL': 0, 'L': 7, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'D', 'cuts': {'XXL': 0, 'L': 8, 'XL': 0, 'M': 0, 'S': 0}},
 {'stock': 'A', 'cuts': {'XXL': 0, 'L': 0, 'XL': 8, 'M': 0, 'S': 0}},
 {'stock': 'B', 'cuts': {'XXL': 0, 'L': 0, 'XL': 8, 'M': 0, 'S': 0}},
 {'stock': 'C', 'cuts': {'XXL': 0, 'L': 0, 'XL': 7, 'M': 0, 'S': 0}},
 {'stock': 'D', 'cuts': {'XXL': 0, 'L': 0, 'XL': 8, 'M': 0, 'S': 0}},
 {'stock': 'A', 'cuts': {'XXL': 0, 'L': 0, 'XL': 0, 'M': 17, 'S': 0}},
 {'stock': 'B', 'cuts': {'XXL': 0, 'L': 0, 'XL': 0, 'M': 17, 'S': 0}},
 {'stock': 'C', 'c

## Optimal cutting using known patterns

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to meet the demand for finished parts at minimum cost.

Let the index $s_p$ denote the stock specified by pattern $p$, and let $x_{s_p}$ denote the number pieces of stock $s_p$ is used. For a given list of patterns, the minimum cost optimization problem is a mixed integer linear optimization (MILO) subject to meeting demand constraints for each finished item.

### Sets and Indices

$s \in \text{Stock}=\{1,2,\dots, N\}$: Set of available stock material units.

$f \in \text{Finished Good}$: Set of final finised goods.

### Decision variables

$ \text{trimloss}_i \in [x \text{ mm}, 4\% \text{ Mother\_Coil\_Width}]:  x (mm) $ the lower bound, depending on the thickness of the Mother Coil if stock $i$ is used; $4\%$ the upper bound .

$\text{cut}_{i,j} \in \mathbb{N}$: Number of pieces of type $j$ cut out from stock material unit $i$.

### Objective Functions

- **Material Consumption:** Minimize the number of stock material units used.

$$
\begin{equation}
\text{Min} \sum_{i \in \text{Stock}}{\text{trimloss}_i}
\tag{1.0}
\end{equation}
$$

### Constraints


#### Weight Constraints
- **Satisfy Demand:** Total weight of final pieces of type $j$ obtained must satisfy its demand requirement.

$$
\begin{equation}
\text{Upper bound weight}_j \geq P{p \in \text{Patterns}}{\text{ Cut}_{p,j} \cdot \text{Weight per cut}_p}   \quad \forall j \in \text{Finished Goods}
\tag{2.1}
\end{equation}
$$



The following cell is an AMPL implementation of this optimization model.